In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
Rootdf = pd.read_csv('Root_Insurance_data_Edited.csv')


Rootdf['Multi_VD'] = Rootdf.Num_Vehicles * Rootdf.Num_Drivers
Rootdf['Add_VD'] = Rootdf.Num_Vehicles + Rootdf.Num_Drivers

Rootdf = Rootdf.replace('Y',1)
Rootdf = Rootdf.replace('N',0)
Rootdf = Rootdf.replace('U',2)
Rootdf = Rootdf.replace('S',0)
Rootdf = Rootdf.replace('M',1)


#Rootdf.Insured[Rootdf.Insured == 'N'] = 0
#Rootdf.Insured[Rootdf.Insured == 'Y'] = 1
#Rootdf.Insured[Rootdf.Insured == 'U'] = 2

#Rootdf.Marital[Rootdf.Marital == 'S'] = 0
#Rootdf.Marital[Rootdf.Marital == 'M'] = 1

#Rootdf.Click[Rootdf.Click == 'N'] = 0
#Rootdf.Click[Rootdf.Click == 'Y'] = 1

In [3]:
Rootdf.head()

,Insured,Num_Vehicles,Num_Drivers,Marital,Bid,Rank,Click,Policies_sold,Multi_VD,Add_VD
0,2,2,1,1,10,2,0,0,2,3
1,1,3,1,1,10,5,0,0,3,4
2,1,3,2,1,10,5,0,0,6,5
3,0,3,2,0,10,4,0,0,6,5
4,2,2,2,0,10,2,0,0,4,4


In [4]:
X = Rootdf[["Insured", "Num_Vehicles", "Num_Drivers", "Marital","Multi_VD","Add_VD"]]
Y = Rootdf.Rank
X.head()

,Insured,Num_Vehicles,Num_Drivers,Marital,Multi_VD,Add_VD
0,2,2,1,1,2,3
1,1,3,1,1,3,4
2,1,3,2,1,6,5
3,0,3,2,0,6,5
4,2,2,2,0,4,4


In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2,
                                                    random_state = 314,
                                                    shuffle=True,
                                                    stratify=Rootdf[['Rank','Click','Policies_sold']])

In [6]:
#print(x_train.Rank.value_counts()/x_train.Rank.count())
#print(x_test.Rank.value_counts()/x_test.Rank.count())

#print(x_train.Click.value_counts()/x_train.Click.count())
#print(x_test.Click.value_counts()/x_test.Click.count())

print(y_train.value_counts()/y_train.count())
print(y_test.value_counts()/y_test.count())

3    0.240125
5    0.229125
4    0.208875
1    0.161000
2    0.160875
Name: Rank, dtype: float64
3    0.2400
5    0.2290
4    0.2090
1    0.1615
2    0.1605
Name: Rank, dtype: float64


In [17]:
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import auc, roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

def powerset(s):
    power_set = []
    x = len(s)
    for i in range(1 << x):
        power_set.append([s[j] for j in range(x) if (i & (1 << j))])
        
    return power_set[1:]

In [22]:
## get all models we're interested in
models = powerset(["Insured", "Num_Vehicles", "Num_Drivers", "Marital","Multi_VD","Add_VD"])
models.append("baseline")

skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state = 5309)

## make empty mse holder
cv_roc_auc_ovo_model = np.zeros((5, len(models)))
cv_roc_auc_ovr_model = np.zeros((5, len(models)))
cv_acc_score_model = np.zeros((5, len(models)))

## make base regression object
LR = LogisticRegression(max_iter=1000)

## loop through all splits
i = 0
for train_index, test_index in skfold.split(x_train,y_train):
    ## get train and holdout sets
    x_train_train = x_train.iloc[train_index]
    x_holdout = x_train.iloc[test_index]
    y_train_train = y_train.iloc[train_index]
    y_holdout = y_train.iloc[test_index]

    ## loop through all models
    j = 0
    for model in models:
        if model == "baseline":
            y_mean_proba = y_train_train.value_counts(normalize=True,sort=False).sort_index().array
            y_mean_proba_array = np.ones((len(y_holdout),5))
            y_mean = 3
            y_mean_array = y_mean*np.ones(len(y_holdout))
            for k in range(0,len(y_holdout)):
                y_mean_proba_array[k]=y_mean_proba
            cv_roc_auc_ovo_model[i,j] = roc_auc_score(y_holdout,y_mean_proba_array,multi_class='ovo')
            cv_roc_auc_ovr_model[i,j] = roc_auc_score(y_holdout,y_mean_proba_array,multi_class='ovr')
            cv_acc_score_model[i,j] = accuracy_score(y_holdout,y_mean_array)

        else:
            ## make clone
            LR_clone = clone(LR)
        
        
            ## fit clone
            LR_clone.fit(x_train_train[model],y_train_train)

            y_proba_predict = LR_clone.predict_proba(x_holdout[model])

            ## record mse
            cv_roc_auc_ovo_model[i,j] = roc_auc_score(y_holdout,y_proba_predict,multi_class='ovo')
            cv_roc_auc_ovr_model[i,j] = roc_auc_score(y_holdout,y_proba_predict,multi_class='ovr')
            cv_acc_score_model[i,j] = accuracy_score(y_holdout,LR_clone.predict(x_holdout[model]))
        j=j+1
    i=i+1  

In [23]:
models

[['Insured'],
 ['Num_Vehicles'],
 ['Insured', 'Num_Vehicles'],
 ['Num_Drivers'],
 ['Insured', 'Num_Drivers'],
 ['Num_Vehicles', 'Num_Drivers'],
 ['Insured', 'Num_Vehicles', 'Num_Drivers'],
 ['Marital'],
 ['Insured', 'Marital'],
 ['Num_Vehicles', 'Marital'],
 ['Insured', 'Num_Vehicles', 'Marital'],
 ['Num_Drivers', 'Marital'],
 ['Insured', 'Num_Drivers', 'Marital'],
 ['Num_Vehicles', 'Num_Drivers', 'Marital'],
 ['Insured', 'Num_Vehicles', 'Num_Drivers', 'Marital'],
 ['Multi_VD'],
 ['Insured', 'Multi_VD'],
 ['Num_Vehicles', 'Multi_VD'],
 ['Insured', 'Num_Vehicles', 'Multi_VD'],
 ['Num_Drivers', 'Multi_VD'],
 ['Insured', 'Num_Drivers', 'Multi_VD'],
 ['Num_Vehicles', 'Num_Drivers', 'Multi_VD'],
 ['Insured', 'Num_Vehicles', 'Num_Drivers', 'Multi_VD'],
 ['Marital', 'Multi_VD'],
 ['Insured', 'Marital', 'Multi_VD'],
 ['Num_Vehicles', 'Marital', 'Multi_VD'],
 ['Insured', 'Num_Vehicles', 'Marital', 'Multi_VD'],
 ['Num_Drivers', 'Marital', 'Multi_VD'],
 ['Insured', 'Num_Drivers', 'Marital', 'Mult

In [24]:
ovo_max = np.argmax(np.mean(cv_roc_auc_ovo_model,axis=0))
ovr_max = np.argmax(np.mean(cv_roc_auc_ovr_model,axis=0))
acc_max = np.argmax(np.mean(cv_acc_score_model,axis=0))

print("The Model with Max ROC AUC OVO is ",models[ovo_max])
print("The Model with Max ROC AUC OVR is ",models[ovr_max])
print("The Model with Max Accuracy is ",models[acc_max])
print("For Model:",models[ovo_max],"\nThe OVO ROC score is ",np.mean(cv_roc_auc_ovo_model,axis=0)[ovo_max],
      #"with standard deviation of",np.std(cv_roc_auc_ovo_model,axis=0)[ovo_max],
      "\nThe OVR ROC score is ",np.mean(cv_roc_auc_ovr_model,axis=0)[ovr_max],
      #"with standard deviation of",np.std(cv_roc_auc_ovo_model,axis=0)[ovr_max],
      "\nThe Accuracy score is ",np.mean(cv_acc_score_model,axis=0)[acc_max]
)

The Model with Max ROC AUC OVO is  ['Insured', 'Num_Drivers', 'Marital', 'Multi_VD']
The Model with Max ROC AUC OVR is  ['Insured', 'Num_Drivers', 'Marital', 'Multi_VD']
The Model with Max Accuracy is  ['Insured', 'Num_Drivers', 'Multi_VD']
For Model: ['Insured', 'Num_Drivers', 'Marital', 'Multi_VD'] 
The OVO ROC score is  0.7305783956558944 
The OVR ROC score is  0.7265655379850982 
The Accuracy score is  0.394875


In [25]:
for i in range(0,len(models)):
    if np.mean(cv_roc_auc_ovo_model,axis=0)[i] > 0.730 :
        print("For Model:",models[i],
              "\nThe OVO ROC score is ",np.mean(cv_roc_auc_ovo_model,axis=0)[i],
              "\nThe OVR ROC score is ",np.mean(cv_roc_auc_ovr_model,axis=0)[i],
              "\nThe Accuracy score is ",np.mean(cv_acc_score_model,axis=0)[i])

For Model: ['Insured', 'Num_Drivers', 'Multi_VD'] 
The OVO ROC score is  0.7301317427654155 
The OVR ROC score is  0.7261433104641407 
The Accuracy score is  0.394875
For Model: ['Insured', 'Num_Drivers', 'Marital', 'Multi_VD'] 
The OVO ROC score is  0.7305783956558944 
The OVR ROC score is  0.7265655379850982 
The Accuracy score is  0.394875


In [211]:
for i in range(0,len(models)):
    print("For Model:",models[i],"\nThe OVO ROC score is ",np.mean(cv_roc_auc_ovo_model,axis=0)[i],
          "with standard deviation of ",np.std(cv_roc_auc_ovo_model,axis=0)[i],
          "\nThe OVR ROC score is ",np.mean(cv_roc_auc_ovr_model,axis=0)[i],
          "with standard deviation of ",np.std(cv_roc_auc_ovr_model,axis=0)[i],
)


For Model: ['Insured'] 
The OVO ROC score is  0.6771145832482947 with standard deviation of  0.006937712501053638 
The OVR ROC score is  0.6706935192883458 with standard deviation of  0.005477795352796611
For Model: ['Num_Vehicles'] 
The OVO ROC score is  0.6239994663101864 with standard deviation of  0.0075148641270486325 
The OVR ROC score is  0.6251983476353452 with standard deviation of  0.007383865036054034
For Model: ['Insured', 'Num_Vehicles'] 
The OVO ROC score is  0.7489161633739552 with standard deviation of  0.004529247948019469 
The OVR ROC score is  0.7469316537132185 with standard deviation of  0.004172767080933091
For Model: ['Num_Drivers'] 
The OVO ROC score is  0.5187987093828218 with standard deviation of  0.007143889341825646 
The OVR ROC score is  0.518899015560381 with standard deviation of  0.007252250494781755
For Model: ['Insured', 'Num_Drivers'] 
The OVO ROC score is  0.6951558575146377 with standard deviation of  0.006783659045852076 
The OVR ROC score is  0.6

In [212]:
## get all models we're interested in
model = ["Insured", "Num_Vehicles", "Num_Drivers"]#,"Multi_VD","Add_VD"])
#model = "baseline"

skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state = 5309)
depth = 20

## make empty mse holder
cv_roc_auc_ovo_depth = np.zeros((5, depth))
cv_roc_auc_ovr_depth = np.zeros((5, depth))

## loop through all splits
i = 0
for train_index, test_index in skfold.split(x_train,y_train):
    ## get train and holdout sets
    x_train_train = x_train.iloc[train_index]
    x_holdout = x_train.iloc[test_index]
    y_train_train = y_train.iloc[train_index]
    y_holdout = y_train.iloc[test_index]

    for j in range(1,20):
        tree_clf = DecisionTreeClassifier(max_depth = j)
        if model == "baseline":
            y_mean_proba = y_train_train.value_counts(normalize=True,sort=False).sort_index().array
            y_mean_proba_array = np.ones((len(y_holdout),5))
            for k in range(0,len(y_holdout)):
                y_mean_proba_array[k]=y_mean_proba
            cv_roc_auc_ovo_depth[i,j] = roc_auc_score(y_holdout,y_mean_proba_array,multi_class='ovo')
            cv_roc_auc_ovr_depth[i,j] = roc_auc_score(y_holdout,y_mean_proba_array,multi_class='ovr')

        else:
            ## make clone
            tree_clone = clone(tree_clf)
                
            ## fit clone
            tree_clone.fit(x_train_train[model],y_train_train)

            y_proba_predict = tree_clone.predict_proba(x_holdout[model])

            ## record mse
            cv_roc_auc_ovo_depth[i,j] = roc_auc_score(y_holdout,y_proba_predict,multi_class='ovo')
            cv_roc_auc_ovr_depth[i,j] = roc_auc_score(y_holdout,y_proba_predict,multi_class='ovr')
    i=i+1  

In [214]:
print(np.mean(cv_roc_auc_ovo_depth,axis=0))
print(np.mean(cv_roc_auc_ovr_depth,axis=0))

[0.         0.67303032 0.74206607 0.75021254 0.75187168 0.75429773
 0.75429773 0.75429773 0.75429773 0.75429773 0.75429773 0.75429773
 0.75429773 0.75429773 0.75429773 0.75429773 0.75429773 0.75429773
 0.75429773 0.75429773]
[0.         0.66515946 0.73789133 0.7481348  0.75049283 0.75335697
 0.75335697 0.75335697 0.75335697 0.75335697 0.75335697 0.75335697
 0.75335697 0.75335697 0.75335697 0.75335697 0.75335697 0.75335697
 0.75335697 0.75335697]


In [234]:
tree_clf = DecisionTreeClassifier(max_depth = 6)
tree_clf.fit(x_train,y_train)
            
print(roc_auc_score(y_train,tree_clf.predict_proba(x_train),multi_class='ovo'))
print(roc_auc_score(y_train,tree_clf.predict_proba(x_train),multi_class='ovr'))
pd.DataFrame(confusion_matrix(y_train, tree_clf.predict(x_train))/y_train.count())


0.7651760372910791
0.7643842489264085


,0,1,2,3,4
0,0.116125,0.016750,0.028125,0.000000,0.000000
1,0.070375,0.027875,0.047000,0.015625,0.000000
2,0.044000,0.025750,0.065875,0.026125,0.078375
3,0.000000,0.015125,0.044750,0.028750,0.120250
4,0.000000,0.000000,0.012000,0.018750,0.198375


In [203]:
roc_auc_score(y_train,tree_clf.predict_proba(x_train),multi_class='ovo')

0.7651760372910791

In [204]:
roc_auc_score(y_train,tree_clf.predict_proba(x_train),multi_class='ovr')

0.7643842489264085

In [205]:
y_mean_proba = y_train.value_counts(normalize=True,sort=False).sort_index().array

In [206]:
y_mean_proba 
y_mean_proba_array = np.ones((len(y_train),5))

for i in range(0,len(y_train)):
    y_mean_proba_array[i]=y_mean_proba
 